In [1]:
from elasticsearch import Elasticsearch, exceptions
from agents import function_tool
from typing import List, Dict

In [2]:
es = Elasticsearch(hosts="http://localhost:9200")

In [5]:
@function_tool
def search_videos(query: str, index_name: str="podcasts", size: int = 5) -> list[dict]:
    """
    Performs a full-text search across video titles and subtitles using Elasticsearch.
    
    This function utilizes a 'multi_match' query with a 'best_fields' type. It prioritizes 
    matches found in the title over subtitles and applies a custom English analyzer.

    Args:
        query (str): The search terms provided by the user.
        index_name (str, optional): The Elasticsearch index to query. 
            Defaults to "podcasts".
        size (int, optional): The maximum number of search results to return. 
            Defaults to 5.

    Returns:
        list: A list of dictionaries containing highlighted snippets and the 
            associated 'video_id'. Returns an empty list if an error occurs.

    Example:
        >>> results = search_videos("machine learning", size=1)
        >>> print(results)
        [
            {
                'title': ['Intro to *Machine Learning*'],
                'subtitles': ['In this video, we discuss *machine learning* basics...'],
                'video_id': 'vid_001'
            }
        ]

    Raises:
        ElasticsearchException: Logged internally, returns empty list on failure.
    """
    
    body = {
        "size": size,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "subtitles"],
                "type": "best_fields",
                "analyzer": "my_english_analyzer"
            }
        },
        "highlight": {
            "pre_tags": ["*"],
            "post_tags": ["*"],
            "fields": {
                "title": {"fragment_size": 150, "number_of_fragments": 1},
                "subtitles": {
                    "fragment_size": 150, 
                    "number_of_fragments": 3, 
                    "order": "score"
                }
            }
        }
    }
    
    try:
        response = es.search(index=index_name, body=body)
        hits = response.body['hits']['hits']
        
        results = []
        for hit in hits:
            # Safely get highlights; default to empty dict if no matches found in fields
            highlight = hit.get('highlight', {})
            highlight['video_id'] = hit['_id']
            results.append(highlight)
            return results

    except exceptions.NotFoundError:
        print(f"Error: Index '{index_name}' not found.")
    except exceptions.ConnectionError:
        print("Error: Could not connect to Elasticsearch.")
    except exceptions.RequestError as e:
        print(f"Error: Invalid search request. {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    
    return []

In [46]:
from agents import Agent, Runner, ItemHelpers
from config import research_instructions

In [7]:
faq_instructions = research_instructions
faq_agent = Agent(
    name="faq_assistant",
    tools=[search_videos],
    instructions=faq_instructions,
    model="gpt-4o-mini"
)
    

In [8]:
result = await Runner.run(faq_agent, "Please explain how to manage mental health in kids?")

In [9]:
print(result.final_output)

### Key Concepts for Managing Mental Health in Kids

1. **Early Socialization**:
   - It's important for children to socialize from a young age, as this fosters emotional connections and helps in their brain development, which is essential for mental health [Video ID: C-1Ukfaf7co, 42:59].

2. **Understanding Mental Health Challenges**:
   - Many children are diagnosed with mental health conditions such as ADHD. Recognizing these conditions allows for proper support and interventions [Video ID: tool-R8VJ2Y, 11:32].

3. **Setting Boundaries**:
   - Teaching kids how to acknowledge their emotions and set boundaries is crucial. This includes knowing how to express anger appropriately and manage expectations within relationships [Video ID: L7zWT3l3DV0, 50:50].

4. **Play as a Therapeutic Tool**:
   - Engaging in play is not just fun; it’s essential for emotional and mental health. Play helps children express themselves and process their experiences [Video ID: tool-R8VJ2Y, 1:10:02].

5. **Tr

In [10]:
result = await Runner.run(faq_agent, "How do I cook pizza?")

In [11]:
print(result.final_output)

I wasn't able to find specific video transcripts on cooking pizza. However, I can guide you through some common steps involved in making pizza:

### Basic Steps to Cook Pizza

1. **Gather Ingredients**:
   - **Dough**: You can either make your own or buy pre-made pizza dough.
   - **Sauce**: Tomato sauce, pesto, or even a white sauce.
   - **Cheese**: Mozzarella is the classic choice, but you can use others as well.
   - **Toppings**: Pepperoni, vegetables, herbs, etc., based on your preference.

2. **Preheat the Oven**:
   - Set your oven to a high temperature, usually around 475°F (245°C).

3. **Prepare the Dough**:
   - If using homemade dough, roll it out to your desired thickness.
   - For pre-made dough, follow the instructions for prep time.

4. **Add Sauce and Toppings**:
   - Spread a thin layer of sauce over the base.
   - Sprinkle cheese, followed by your chosen toppings.

5. **Bake**:
   - Place the pizza on a baking sheet or pizza stone.
   - Bake in the preheated oven for

In [12]:
from pydantic import BaseModel

In [13]:
class GuardrailOutput(BaseModel):
    reasoning: str
    fail: bool

In [14]:
topic_guardrail_instructions = """
You are a topic guardrail for a self improvement mental wellbeing assistant.

Your job is to check if the user's question is related to:
- Health and Wellbeing
- ADHD
- Mental Health
- Trauma

If the question is about these topics, set fail=False.

If it's about something unrelated (like cooking, sports, celebrity gossip, medical advice, etc.), set fail=True.
If the question contains any abusive, sexual or toxic words, set fail=True.
If the question is combined with any topics out of scope or unrelated, set fail=True.
If the question is combined with any abusive, sexual or toxic words even though is related, set fail=True.
Keep your reasoning under 15 words.
""".strip()

output_guardrail_instructions = """
You are a output guardrail for a self improvement mental wellbeing assistant.

Your job is to check if the answer is related to:
- Health and Wellbeing
- ADHD
- Mental Health
- Trauma

If the output is about these topics, set fail=False.

If it's about something unrelated (like cooking, sports, celebrity gossip, medical advice, etc.), set fail=True.
If it's abusive or toxic or hallucinated, set fail=True.
If the output is combined with any topics out of scope, set fail=True.

Keep your reasoning under 15 words.
"""

topic_guardrail_agent = Agent(
    name="topic_guardrail",
    instructions=topic_guardrail_instructions,
    model="gpt-4o-mini",
    output_type=GuardrailOutput,
)

safety_guardrail_agent = Agent(
    name="safety_guardrail",
    instructions=output_guardrail_instructions,
    model="gpt-4o-mini",
    output_type=GuardrailOutput,
)


In [15]:
result = await Runner.run(topic_guardrail_agent, "How do I install Docker?")
print(f"Relevant: {result.final_output}")

Relevant: reasoning='Question is related to technology, not mental wellbeing.' fail=True


In [16]:
from agents import input_guardrail, GuardrailFunctionOutput, output_guardrail
from agents.exceptions import InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered

In [17]:
@input_guardrail
async def topic_guardrail(ctx, agent, input_info):
    """Check if question is about Health and wellbeing, and self improvement"""
    result = await Runner.run(topic_guardrail_agent, input_info)
    output = result.final_output
    return GuardrailFunctionOutput(
        output_info=output.reasoning,
        tripwire_triggered=output.fail
    )

In [18]:
@output_guardrail
async def safety_guardrail(ctx, agent, agent_output):
    """Check if the response is about Health and wellbeing, and self improvement"""
    guardrail_input = f"Agent responded with {agent_output}"
    result = await Runner.run(safety_guardrail_agent, guardrail_input)
    output = result.final_output
    return GuardrailFunctionOutput(
        output_info=output.reasoning,
        tripwire_triggered=output.fail
    )

In [19]:
guarded_faq_agent = Agent(
    name="guarded_faq_assistant",
    instructions=faq_instructions,
    tools=[search_videos],
    model="gpt-4o-mini",
    input_guardrails=[topic_guardrail],
    output_guardrails=[safety_guardrail]
)

In [20]:
async def run_with_guardrail(user_input: str):
    try:
        result =  await Runner.run(guarded_faq_agent, input=user_input)
        print(result.final_output)
        return result.final_output
    except InputGuardrailTripwireTriggered as e:
        print (f"[BLOCKED] {e.guardrail_result.output.output_info}")
        return None
    except OutputGuardrailTripwireTriggered as e:
        print (f"[BLOCKED] {e.guardrail_result.output.output_info}")
        return None

In [21]:
await run_with_guardrail("Can you help me make a pizza, since I have ADHD, how to manage that?")

[BLOCKED] The question includes cooking, which is unrelated to ADHD management.


In [22]:
await run_with_guardrail("Why kids with ADHD are mean or pricks, how to manage that?")

Managing children with ADHD who display challenging behaviors can be complex, but understanding the underlying issues can help caregivers and educators implement effective strategies. Here are key insights gathered from various sources regarding why children with ADHD may seem "mean" or aggressive, along with management strategies:

### Understanding the Behavior

1. **Emotional Dysregulation**: Children with ADHD often struggle with emotional regulation. They may react with anger or aggression due to their inability to manage frustration or overwhelming emotions. Emotional outbursts can be mistaken for meanness, but they stem from a place of distress (source: [Video](https://youtu.be/tool-R8VJ2Y?t=41)).

2. **Social Skills Deficits**: Many kids with ADHD have difficulties in social situations, leading to misunderstandings and conflict with peers. They may misread social cues or struggle to maintain friendships, which could lead to negative perceptions from others (source: [Video](http

'Managing children with ADHD who display challenging behaviors can be complex, but understanding the underlying issues can help caregivers and educators implement effective strategies. Here are key insights gathered from various sources regarding why children with ADHD may seem "mean" or aggressive, along with management strategies:\n\n### Understanding the Behavior\n\n1. **Emotional Dysregulation**: Children with ADHD often struggle with emotional regulation. They may react with anger or aggression due to their inability to manage frustration or overwhelming emotions. Emotional outbursts can be mistaken for meanness, but they stem from a place of distress (source: [Video](https://youtu.be/tool-R8VJ2Y?t=41)).\n\n2. **Social Skills Deficits**: Many kids with ADHD have difficulties in social situations, leading to misunderstandings and conflict with peers. They may misread social cues or struggle to maintain friendships, which could lead to negative perceptions from others (source: [Vide

In [23]:
from asyncio import gather, tasks, create_task
import asyncio

In [24]:
async def mock_guardrail(input: str) -> str:
    """Function that mocks guardrail"""

    print(f"[Guardrail] starting work on: {input}")
    await asyncio.sleep(2)
    print(f"[Guardrail] good")
    return f"Response to: {input}"

In [25]:
async def mock_guardrail_with_failure(input: int) -> str:
    """Function that mocks guardrail"""
    try:
        print(f"[Guardrail] starting work on: {input}")
        await asyncio.sleep(input)
        print(f"[Guardrail] good")
        return f"Response to: {input}"
    except TypeError as e:
        raise e

In [26]:
async def mock_agent(input: str) -> str:
    """Function that mocks guardrail"""

    print(f"[Agent] starting work on: {input}")
    await asyncio.sleep(2)
    print(f"[Agent] Done")
    return f"Response to: {input}"

In [27]:
# Bad idea
guardrail_result = await mock_guardrail("Hello")
agent_result = await mock_agent("Hello")

[Guardrail] starting work on: Hello
[Guardrail] good
[Agent] starting work on: Hello
[Agent] Done


In [28]:
# Slightly better, agent can still continue despite guardrail is stuffed
results = await gather(
    mock_guardrail("Hello"),
    mock_agent("Hello")
)

[Guardrail] starting work on: Hello
[Agent] starting work on: Hello
[Guardrail] good
[Agent] Done


In [29]:
# Slightly better, agent can still continue despite guardrail is stuffed
results = await gather(
    mock_guardrail_with_failure("Hello"),
    mock_agent("Hello")
)

[Guardrail] starting work on: Hello
[Agent] starting work on: Hello


TypeError: '<=' not supported between instances of 'str' and 'int'

[Agent] Done


In [30]:
guard_task = create_task(mock_guardrail("Hello"))
agent_task = create_task(mock_agent("Hello"))

try:
    await gather(guard_task, agent_task)
except TypeError:
    agent_task.cancel()
    try:
        await agent_task
    except asyncio.CancelledError:
        print("Agent has been cancelled - saved tokens!")

[Guardrail] starting work on: Hello
[Agent] starting work on: Hello
[Guardrail] good
[Agent] Done


In [31]:
guard_task = create_task(mock_guardrail_with_failure("Hello"))
agent_task = create_task(mock_agent("Hello"))

try:
    await gather(guard_task, agent_task)
except TypeError as e:
    print(e)
    agent_task.cancel()
    try:
        await agent_task
    except asyncio.CancelledError:
        print("Agent has been cancelled - saved tokens!")

[Guardrail] starting work on: Hello
[Agent] starting work on: Hello
'<=' not supported between instances of 'str' and 'int'
Agent has been cancelled - saved tokens!


In [82]:

result = Runner.run_streamed(guarded_faq_agent, user_input)
async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

Agent updated: guarded_faq_assistant
-- Tool was called
-- Tool output: [{'subtitles': ['Cuz all\n41:49 three of my children have *ADHD*.\n41:52 Well, I I can tell you what I The first\n41:55 book I ever wrote, Scattered Minds, was\n41:56 an *ADHD*', 'That\n36:55 created hypervigilance and anxiety and\n36:57 probably *ADHD*.', 'So\n41:39 people, kids whose mothers were\n41:42 depressed postpartum have a higher risk\n41:44 of\n41:45 *ADHD*.'], 'video_id': 'tool-R8VJ2Y'}]
-- Tool was called
-- Tool output: [{'subtitles': ["ago to 2100 people yeah\n3:37 and you didn't feel you did your best\n3:38 you went home that night what was going\n3:40 on in your head what are the *symptoms*", "self-doubt\n3:24 evidently I'm not um so that's what\n3:28 happened with you say um you let it\n3:31 Cloud you're thinking what are this what\n3:32 were the *symptoms*", 'none of those\n1:12:45 things he just had trouble of\n1:12:46 concentrating and paying attention\n1:12:48 because of all the\n1:12:49 stres

# Now we will update the agent and reduce the tokens

In [116]:
from ai_agents import summarization_agent
from agents import ModelSettings

In [117]:
def get_subtitles_by_id(video_id: str) -> dict:
    """Function to help receive video transcripts from the elasticsearch

     Args:
        video_id (str): The youtube video id for which user can request subtitles"

    Returns:
        dict: A dictionary with video id, title of the video and its subtitles
    """

    result = es.get(index="podcasts", id=video_id)
    return result["_source"]

In [118]:
@function_tool
async def summarize_video_content(video_id: str) -> str:
    """
    Extracts transcript for a given video ID and uses a sub-agent to summarize it.
    Returns the summary.
    """
    print(f"\n[System] Fetching and summarizing video: {video_id}")
    
    # 1. Extract Text (Logic handled here or by another tool)
    raw_text = get_subtitles_by_id(video_id)
    
    # 2. Call the Sub-Agent
    # We run the sub-agent independently to process this specific chunk of text
    result = await Runner.run(summarization_agent, input=f"Transcript: {raw_text}")
    
    # 3. Return the sub-agent's final output to the Main Agent
    return result.final_output

In [119]:
main_faq_agent = Agent(
    name="main_orchestrator_agent",
    instructions=faq_instructions,
    model_settings=ModelSettings(parallel_tool_calls=True),
    tools=[search_videos, summarize_video_content],
    model="gpt-4o-mini",
    input_guardrails=[topic_guardrail],
    output_guardrails=[safety_guardrail]
)

In [103]:
result = Runner.run_streamed(main_faq_agent, user_input, )
async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

Agent updated: main_orchestrator_agent
-- Tool was called
-- Tool output: [{'subtitles': ['Cuz all\n41:49 three of my children have *ADHD*.\n41:52 Well, I I can tell you *what* I The first\n41:55 book I ever wrote, Scattered Minds, was\n41:56 an *ADHD*', "It's\n0:02 *what* h *what* *what* happened inside of you\n0:04 as a result of *what* happened to you.\n0:05 Physical abuse, sexual abuse, emotional\n0:08 abuse of", "It's\n15:06 *what* h *what* happened inside of you as a\n15:08 result of *what* happened to you."], 'video_id': 'tool-R8VJ2Y'}]
-- Tool was called
-- Tool output: [{'subtitles': ["ago to 2100 people yeah\n3:37 and you didn't feel you did your best\n3:38 you went home that night what was going\n3:40 on in your head what are the *symptoms*", "self-doubt\n3:24 evidently I'm not um so that's what\n3:28 happened with you say um you let it\n3:31 Cloud you're thinking what are this what\n3:32 were the *symptoms*", 'none of those\n1:12:45 things he just had trouble of\n1:12:46 co

In [109]:
async def main_stream(user_input: str) -> str:

    result = Runner.run_streamed(main_faq_agent, user_input, max_turns=20)
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types
    
    print("=== Run complete ===")
    return result.final_output

In [108]:
await main_stream("Please explain ADHD")

Agent updated: main_orchestrator_agent
-- Tool was called
-- Message output:
 THOUGHT: Start with broad searches to identify videos about ADHD basics, symptoms, causes, diagnosis, and treatment. I'll run 4 searches across the transcripts index.
-- Tool output: [{'subtitles': ["okay so if that comes up for you then\n1:25:53 the second question would be *what's*\n1:25:55 stopping you\n1:25:58 uh *what's* you know i think *what's*\n1:26:", "or explain\n1:09:15 *what* uh *what* you introduce\n1:09:17 so i've never studied it though that\n1:09:18 much\n1:09:19 i don't know the\n1:09:21 patience for", "happening *what's* driving it *what*\n1:05:28 are the social cultural dynamics\n1:05:30 that are\n1:05:31 driving so much dysfunction\n1:05:34 and it's not that"], 'video_id': 'hhhTWYDPAXI'}]
-- Tool was called
-- Tool output: [{'subtitles': ['0:00 70% of the *adult* population is at least\n0:02 on one medication quarter of women are\n0:04 on anti-depressants the rate of\n0:06 childhood is goi

'THOUGHT:\n- The user asks broadly: "Please explain ADHD." I should give a clear, concise overview (definition, core symptoms, causes, diagnosis, treatments, prognosis) and — per my role — back claims with transcript citations from the video library.\n\nACTION:\n- I searched the transcript index for ADHD-related content and inspected relevant transcripts (selected videos and snippets shown below).\n\nOBSERVATION (key transcripts I used)\n- Video tool-R8VJ2Y — snippets referencing “attention deficit … disorder” and links between early experience/trauma and later symptoms (e.g., mentions around 11:32 and 36:55). (See citation examples below.)  \n- Video L7zWT3l3DV0 — discusses executive function and broader mental-health context (snippets at 0:00 and 6:36, and a discussion including “diagnosis” later).  \n- Video hhhTWYDPAXI — discusses genetics/genes and mental-health risk including ADHD (snippet around 44:57) and behavioral/parenting approaches (around 1:03:19).\n\nFINAL ANSWER — conci

In [120]:
async def main(user_input: str) -> str:

    result = await Runner.run(main_faq_agent, user_input, max_turns=20)
    print("=== Run complete ===")
    return result.final_output

In [122]:
result = await main("Please explain ADHD")


[System] Fetching and summarizing video: tool-R8VJ2Y

[System] Fetching and summarizing video: L7zWT3l3DV0
=== Run complete ===


In [123]:
print(result)

**Attention Deficit Hyperactivity Disorder (ADHD)** is a neurodevelopmental disorder characterized by patterns of inattention, hyperactivity, and impulsivity that interfere with functioning or development. Understanding ADHD involves exploring its symptoms, impacts, and underlying causes. 

### Key Concepts and Symptoms:

1. **Diagnosis and Criteria**:
   - ADHD is diagnosed based on the presence of symptoms that significantly impair functioning in multiple settings (e.g., home, school, work). The symptoms include inattention (difficulty sustaining attention, forgetfulness), hyperactivity (excessive movement, inability to remain seated), and impulsivity (making decisions without considering consequences).

2. **Childhood Development**:
   - Dr. Gabor Maté emphasizes the role of childhood experiences in shaping behavior and emotional states. He notes that children exposed to trauma or who do not receive emotional validation are more likely to develop symptoms of ADHD (Maté, **41:39**) a